In [22]:
#!/usr/bin/python3

from login_details import engine1
from pandas import DataFrame
import pandas as pd

In [23]:
# get data from stoppage_id db
with engine1.connect() as con:
    stoppage_hist_query = con.execute("""
    SELECT id, stoppage_id, datetime
    FROM dg_ln_stoppages_raw
    ORDER BY datetime DESC
    LIMIT 100
    """)
    con.close()

In [24]:
# get data from stoppage_string db
with engine1.connect() as conn:
    stoppage_str_query = conn.execute("""
    SELECT fault_id, fault_str, line_stopped, primary_reason
    FROM dg_stoppage_strings
    ORDER BY fault_id ASC
    """)
    con.close()

In [25]:
# convert database instances to DataFrame

# converting stoppage string instance
stoppage_str_df = DataFrame(stoppage_str_query.fetchall())
stoppage_str_df.columns = stoppage_str_query.keys()
stoppage_str_df.set_index('fault_id')

# converting stoppage history instance
stoppage_history_df = DataFrame(stoppage_hist_query.fetchall())
stoppage_history_df.columns = stoppage_hist_query.keys()


In [26]:
# create the "occurred_at" column with the correct datetime format
stoppage_history_df['occurred_at'] = pd.to_datetime(stoppage_history_df['datetime'].dt.strftime('%d-%m-%Y %H:%M:%S'))

# remove original datetime
del stoppage_history_df['datetime']
del stoppage_history_df['id']

stoppage_history_df.sort_values(by='occurred_at', ascending=False)

,stoppage_id,occurred_at
0,0,2022-01-24 14:39:37
1,1,2022-01-24 14:39:23
2,59,2022-01-24 14:39:21
3,0,2022-01-24 14:23:41
4,1,2022-01-24 14:23:27
...,...,...
95,70,2022-01-24 05:30:35
96,1,2022-01-24 05:29:01
97,60,2022-01-24 05:28:59
98,0,2022-01-24 05:13:27


In [27]:
stoppage_str_df.sort_values(by='fault_id', ascending=True)

,fault_id,fault_str,line_stopped,primary_reason
0,0,Running \ Ready,False,False
1,1,Emergency Stop,True,False
2,2,Low Air Pressure,True,True
3,3,Side B Feeder Inner Ring Inverter Fault,True,True
4,4,Side B Feeder Outer Ring Inverter Fault,True,True
5,5,Side A Feeder Inner Ring Inverter Fault,True,True
6,6,Side A Feeder Outer Ring Inverter Fault,True,True
7,7,Side B Inspection Table Inverter Fault,True,True
8,8,Side A Inspection Table Inverter Fault,True,True
9,9,Box Conveyor Inverter Fault,True,True


In [56]:
for row in stoppage_history_df.rows:
   print(row['stoppage_id'], row['datetime'])


AttributeError: 'DataFrame' object has no attribute 'rows'

In [42]:
for column_id, row_id in stoppage_history_df.iterrows():
    for column_str, row_str in stoppage_str_df.iterrows():
        if row_str['fault_id'] == row_id['stoppage_id']:
            fault = str(row_str[1])

    stoppage_history_df['stoppage_str'] = fault

In [55]:
for column_id, row_id in stoppage_history_df.iterrows():
    for column_str, row_str in stoppage_str_df.iterrows():
        if row_str['fault_id'] == row_id['stoppage_id']:
            fault = row_str[1]

    print(stoppage_history_df)
    stoppage_history_df['stoppage_str'] = fault
    print(stoppage_history_df)

    stoppage_id         occurred_at    stoppage_str
0             0 2022-01-24 14:39:37  Emergency Stop
1             1 2022-01-24 14:39:23  Emergency Stop
2            59 2022-01-24 14:39:21  Emergency Stop
3             0 2022-01-24 14:23:41  Emergency Stop
4             1 2022-01-24 14:23:27  Emergency Stop
..          ...                 ...             ...
95           70 2022-01-24 05:30:35  Emergency Stop
96            1 2022-01-24 05:29:01  Emergency Stop
97           60 2022-01-24 05:28:59  Emergency Stop
98            0 2022-01-24 05:13:27  Emergency Stop
99            1 2022-01-24 05:13:25  Emergency Stop

[100 rows x 3 columns]
    stoppage_id         occurred_at     stoppage_str
0             0 2022-01-24 14:39:37  Running \ Ready
1             1 2022-01-24 14:39:23  Running \ Ready
2            59 2022-01-24 14:39:21  Running \ Ready
3             0 2022-01-24 14:23:41  Running \ Ready
4             1 2022-01-24 14:23:27  Running \ Ready
..          ...                 ..

In [40]:
stoppage_history_df

,stoppage_id,occurred_at,stoppage_str
0,0,2022-01-24 14:39:37,Emergency Stop
1,1,2022-01-24 14:39:23,Emergency Stop
2,59,2022-01-24 14:39:21,Emergency Stop
3,0,2022-01-24 14:23:41,Emergency Stop
4,1,2022-01-24 14:23:27,Emergency Stop
...,...,...,...
95,70,2022-01-24 05:30:35,Emergency Stop
96,1,2022-01-24 05:29:01,Emergency Stop
97,60,2022-01-24 05:28:59,Emergency Stop
98,0,2022-01-24 05:13:27,Emergency Stop


In [31]:
for column, row in stoppage_history_df.iterrows():


    #stoppage_str_df['fault_str']#.loc[stoppage_str_df['fault_id']==row['stoppage_id']]
    
    mask = stoppage_str_df['fault_str'].loc[stoppage_str_df['fault_id']==row['stoppage_id']]

    #mask = (df['Qty'] == 1) & (df['Price'] == 10)

    print('-----')
    print(row['stoppage_id'])
   # print(' ')
   # print(mask[0])
    #print(mask[row['stoppage_id']])

    stoppage_history_df['stoppage_str'] = mask[row['stoppage_id']]

    print(stoppage_history_df)
    #print(row['stoppage_id'])
    #print(stoppage_history_df['stoppage_str'])


-----
0
    stoppage_id         occurred_at     stoppage_str
0             0 2022-01-24 14:39:37  Running \ Ready
1             1 2022-01-24 14:39:23  Running \ Ready
2            59 2022-01-24 14:39:21  Running \ Ready
3             0 2022-01-24 14:23:41  Running \ Ready
4             1 2022-01-24 14:23:27  Running \ Ready
..          ...                 ...              ...
95           70 2022-01-24 05:30:35  Running \ Ready
96            1 2022-01-24 05:29:01  Running \ Ready
97           60 2022-01-24 05:28:59  Running \ Ready
98            0 2022-01-24 05:13:27  Running \ Ready
99            1 2022-01-24 05:13:25  Running \ Ready

[100 rows x 3 columns]
-----
1
    stoppage_id         occurred_at    stoppage_str
0             0 2022-01-24 14:39:37  Emergency Stop
1             1 2022-01-24 14:39:23  Emergency Stop
2            59 2022-01-24 14:39:21  Emergency Stop
3             0 2022-01-24 14:23:41  Emergency Stop
4             1 2022-01-24 14:23:27  Emergency Stop
..          

KeyError: 59

In [189]:
stoppage_history_df

,stoppage_id,occurred_at,stoppage_str
0,0,2022-01-24 14:39:37,NaN
1,1,2022-01-24 14:39:23,Emergency Stop
2,59,2022-01-24 14:39:21,NaN
3,0,2022-01-24 14:23:41,NaN
4,1,2022-01-24 14:23:27,NaN
...,...,...,...
95,70,2022-01-24 05:30:35,NaN
96,1,2022-01-24 05:29:01,NaN
97,60,2022-01-24 05:28:59,NaN
98,0,2022-01-24 05:13:27,NaN


In [108]:
var = stoppage_str_df.loc[5, 'fault_id']
#stoppage_str_df_2['fault_id'] == 0
print(var['fault_str'])

IndexError: invalid index to scalar variable.

In [108]:
#from numpy.compat import basestring

# Iterate through the dataframe

for index_history, stoppage_id in stoppage_history_df.iterrows():
    #stoppage_str_df_2 = stoppage_str_df.loc[stoppage_str_df['fault_id'] == int(stoppage_id[0])]
    print(stoppage_str_df.loc[int(stoppage_id[0]), 'fault_id'])
    #print(stoppage_str_df_2)
    #stoppage_history_df['stoppage_reason'] = stoppage_str_df_2['fault_str']
    #stoppage_history_df['stoppage_reason']= stoppage_str_df_2['fault_str']
    #print(stoppage_history_df['stoppage_reason'])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'Timestamp'

In [63]:
print(stoppage_history_df)

    stoppage_id         occurred_at stoppage_reason
0             0 2021-10-19 06:49:43             NaN
1            64 2021-10-19 06:55:17             NaN
2            49 2021-10-19 06:55:19             NaN
3             1 2021-10-19 06:55:51             NaN
4            59 2021-10-19 06:55:53             NaN
..          ...                 ...             ...
95           59 2021-10-19 10:31:58             NaN
96            0 2021-10-19 10:32:18             NaN
97           42 2021-10-19 10:42:08             NaN
98            0 2021-10-19 10:43:56             NaN
99           44 2021-10-19 10:44:28             NaN

[100 rows x 3 columns]


In [120]:
# print rows based on stoppage_id value
# stoppage_history_df.loc[stoppage_history_df['stoppage_id']==0]

,stoppage_id,occurred_at
0,0,2021-10-19 06:49:43
5,0,2021-10-19 06:55:57
7,0,2021-10-19 07:00:12
10,0,2021-10-19 07:14:04
14,0,2021-10-19 07:15:52
16,0,2021-10-19 07:18:20
18,0,2021-10-19 07:23:56
20,0,2021-10-19 07:30:14
25,0,2021-10-19 07:34:14
31,0,2021-10-19 08:02:46
